# 1. Importar tablas

In [1]:
timeorigin: .z.t

In [2]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [3]:
/ 1.2. Importar tabla de las bases de contribución
contributionBasis:get `:data/contributionBasis;

/show employees

In [4]:
/ 1.3. Importar tabla de tipos
rateTypes:("DDFSsS";enlist csv) 0: `$":data/rateTypes.csv";

/show rateTypes

In [5]:
/ 1.3. Importar tabla de escala irpf
irpfScale:("DDFF";enlist csv)0:`$":data/irpfScale.csv";

irpfScale: ungroup select range, rate, acc:sums prev[rate]*deltas range
                   by startDate, endDate 
                   from irpfScale;

/show irpfScale

# 2. Variables globales

In [6]:
cnaes:string asc distinct rateTypes`cnae;                                 /all the cnae's strings

# 3. Funciones


## 3.1. Cálculo de las aportaciones

In [7]:
gacnae:{`$cnaes last where cnaes in (neg til count string x)_\:string x}; /(g)et (a)ctual (cnae)

gRate:{[mo;conti;contr;cn]                                                /(g)et (rate) type
    0^first exec rate%100 
            from rateTypes 
            where contingency=conti, 
                  contributor=contr, 
                  mo within (startDate;endDate),
                  (cnae=gacnae[cn]) | cnae=`0};

## 3.2. Cálculo de las retenciones

In [8]:
calirpf:{[n;ye]                                                           /(c)alculate (irpf)
    first exec acc+rate*n-range 
          from irpfScale 
          where ye within (`year$startDate;`year$endDate), i=range bin n};

gIrpfRate:{[mo;ni;contr]                                                  /(g)et (irpf) (rate) type
    if[contr=`E;:0];
    r:first exec irpfRate 
           from irpfBasis 
           where nif=ni,year=`year$mo;
    $[r=0;0;r|0.02]};

## 3.3. Funciones para monthlyContributions

In [9]:
gconty:{[bt;ws]                                                           /(g)en (con)tribution (ty)pe
    d: `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase!
    ((`cc;(`at;$[ws=`temp;`ded;`dei];`fp;`fgs);`hef;`he;`irpf;`mei));
    d bt};

cmonthly_contributions:{[nif;mon;contributor;conType;cnae]              /(c)reate monthlyContributions
        $[conType=`irpf;
          :gIrpfRate[mon;nif;contributor];
          :gRate[mon;conType;contributor;cnae]];};

# 4. Generación de las tablas

In [10]:
/---------- monthlyContributionsla auxiliar irpfBasis ----------
irpfBasis: () xkey select aIrpfBase: sum amount, mpf:5500f                /new monthlyContributionsle with: (a)ctual (irpfBase)  & "(m)ínimo (p)ersonal y (f)amiliar"
                   by year:`year$month, nif 
                   from contributionBasis
                   where baseType=`irpfBase;

update retfee: 0 | calirpf'[aIrpfBase;year] - calirpf'[mpf;year]          /add a column for (ret)ention (fee)
    from `irpfBasis;

update irpfRate: 0.01 xbar 0.005 + retfee%aIrpfBase                       /add a column for the irpf Rate
    from `irpfBasis;
/---------- monthlyContributionsla auxiliar irpfBasis ----------


monthlyContributions: update contributionType: gconty'[baseType;workingSchedule]
                      from contributionBasis lj 
                           `nif xkey select workingSchedule, nif from employees;

monthlyContributions: raze {n: 2*count x[`contributionType];
     flip `nif`month`contributor`contributionType`amount!(
        n#x[`nif];
        n#x[`month];
        ((n div 2)#`E),(n div 2)#`T;
        (x[`contributionType]),x[`contributionType];
        n#x[`amount])} each monthlyContributions;

/select from monthlyContributions where nif=`81954661A, month=2024.12m

monthlyContributions: update amount: amount*cmonthly_contributions'[nif;month;contributor;contributionType;cnae]
                      from monthlyContributions lj 
                           `nif xkey select cnae, nif from employees;

monthlyContributions: select nif, month, contributor, contributionType, amount 
                      from `monthlyContributions 
                      where amount<>0;

In [11]:
timeend: .z.t

Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [12]:
show 0^exec `cc`at`fp`fgs`hef`he`irpf`mei#(contributionType!amount)
    by nif,month,contributor
    from monthlyContributions;

nif       month   contributor| cc       at       fp       fgs      hef he    ..
-----------------------------| ----------------------------------------------..
00193031S 2021.07 E          | 247.8    34.65    6.3      2.1      0   0     ..
00193031S 2021.07 T          | 49.35    0        1.05     0        0   0     ..
00193031S 2021.08 E          | 593.7943 83.03056 15.09646 5.032155 0   0     ..
00193031S 2021.08 T          | 118.2556 0        2.516077 0        0   0     ..
00193031S 2021.09 E          | 643.499  89.9808  16.36014 5.453382 0   0     ..
00193031S 2021.09 T          | 128.1545 0        2.726691 0        0   0     ..
00193031S 2021.10 E          | 591.348  82.6885  15.03427 5.011424 0   0     ..
00193031S 2021.10 T          | 117.7685 0        2.505712 0        0   0     ..
00193031S 2022.07 E          | 275.3412 38.5011  7.0002   2.3334   0   0     ..
00193031S 2022.07 T          | 54.8349  0        1.1667   0        0   0     ..
00193031S 2022.08 E          | 639.29   

# 5. Guardar datos en csv

In [13]:
`:data/monthlyContributions set monthlyContributions;

In [14]:
"Number of contribution: ", string count monthlyContributions

"Total time spent: ",string timeend-timeorigin

"Number of contribution: 615896"


"Total time spent: 00:00:07.674"
